In [ ]:
# Imports
import numpy as np
import pandas as pd
import os

# With the toggle '+ Add Input' on the right, add the covid19-radiography-database dataset
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

# **Import dataset**

In [ ]:
from pathlib import Path

# First define your BASE_DIR: this will be the base directory
# in wich you will save your final model
BASE_DIR = Path.cwd()

# Since we are using Kaggle, define the correct path to problem data
dataset_path = Path("/kaggle/input/covid19-radiography-database")
dataset_folder = "COVID-19_Radiography_Dataset"

# Check how the directory is actually organized...
image_dir_names = #CODE

# ...and finally extract data, filling dataset_files
dataset_files = []

#CODE
#CODE
#CODE
#CODE


# How many images do we have?
print(#CODE)

In [ ]:
import math

# Reduce the dimension of the dataset to work faster
new_size = #CODE
dataset_files = #CODE

# How many images do we have in the end?
print(#CODE)

## Check shape of dataset_files

In [ ]:
from PIL import Image

# Before proceding, check whether images and masks have the same size

img = #CODE
mask = #CODE

print(f"image={img.size}, mask={mask.size}")


# **Plot image and mask**

In [ ]:
import matplotlib.pyplot as plt

# Before developing any model, plot the images and masks

img = #CODE
mask = #CODE

plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.title("Image")
plt.imshow(img, cmap="gray")
plt.axis("off")

plt.subplot(1,2,2)
plt.title("Mask")
plt.imshow(mask, cmap="gray")
plt.axis("off")

plt.show()


# **Dataset class**

### Now, implement the Dataset and DataLoader classes, which you will use along the project

In [ ]:
from torch.utils.data import Dataset

# This will be your Dataset class
class SegmentationDataset(Dataset):
    # Initialization of the Dataset
    def __init__(self, data, transform=None):
        #CODE
        #CODE

    def __len__(self):
        #CODE

    def __getitem__(self, idx):
        #CODE
        #CODE
        #CODE
        #CODE

In [ ]:
# This will be your DataLoader class.
# (Actually, we will use the MONAI loader)
import random

class ToyDataLoader:
    def __init__(self, dataset, batch_size=1, shuffle=False):
        #CODE
        #CODE
        #CODE

    def __iter__(self):
        self.indices = #CODE -> set the indices starting from dataset

        #CODE -> shuffle the indices, if necessary

        self.current = #CODE
        return self

    def __next__(self):
        if #CODE:
            raise StopIteration

        batch_indices = #CODE
        self.current += #CODE

        batch = #CODE
        return batch


In [ ]:
# Try your dataset and loader
ds = SegmentationDataset([10, 20, 30, 40, 50])
loader = ToyDataLoader(ds, batch_size=#CODE, shuffle=#CODE)

for batch in loader:
    print(batch)

# Transforms

In [ ]:
# Check https://monai-dev.readthedocs.io/en/stable/transforms.html
from monai.transforms import (
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
)

# Select a size
PATCH_SIZE = #CODE

train_transforms = Compose(
    [
        # First: load the image in dictionary form
        #CODE

        # Then, ensure the correct shape -> (W,H)
        #CODE
        #CODE
        #CODE

        # Now you can use some transformations

        #CODE
        #CODE
        #CODE

        #CODE
        #CODE
        #CODE

        #CODE
        #CODE
        #CODE

        #CODE
        #CODE
        #CODE

        # And finlly transform the image and labels into a tensor
        #CODE
    ]
)

# The same goes for validation transformations:
val_transforms = Compose([
    #CODE
    #CODE
    #CODE

    #CODE
    #CODE
    #CODE

    #CODE
    #CODE
    #CODE
])

In [ ]:
from monai.utils import first, set_determinism

set_determinism(seed=42)

In [ ]:
from monai.data import CacheDataset, DataLoader, decollate_batch

# Here check if your transforms are working correctly, using your Dataset class and
# MONAI's DataLoader
check_dataset = #CODE
check_loader = #CODE
check_data = #CODE
image, label = #CODE
print(f"image shape: {image.shape}, label shape: {label.shape}")

plt.figure("check", (12, 6))
plt.subplot(1, 2, 1)
plt.title("image")
plt.imshow(image[:, :], cmap="gray")
plt.subplot(1, 2, 2)
plt.title("label")
plt.imshow(label[:, :], cmap="gray")
plt.show()

# Model

In [ ]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm
import torch

# Set the device for the model
device = torch.device("cuda:0")

# Now we need a NN for 2D images segmentation:
# we will use a MONAI UNet -> check https://monai-dev.readthedocs.io/en/fixes-sphinx/networks.html
model = UNet(
    # Image dimensions
    spatial_dims= #CODE,
    
    # How many input channels?
    in_channels= #CODE,
    
    # How many output channels? (That is, how many classes to recognize?)
    out_channels= #CODE,
    
    # Feature maps at each level of the net
    channels= #CODE,
    
    # Downsampling factors at each level
    strides= #CODE,

    # Number of residual blocks
    num_res_units= #CODE,

    # Normalization to be used  after convolutions
    norm= #CODE,
).to(device)

## Loss function

In [ ]:
from monai.metrics import #CODE
from monai.losses import #CODE

# Now we need to define a Loss Function: see https://monai.readthedocs.io/en/1.4.0/losses.html
loss_function = #CODE

# Optimizer...
optimizer = #CODE

# ...and here we define a suitble metric to evaluate the segmentation: see https://monai-dev.readthedocs.io/en/fixes-sphinx/metrics.html
dice_metric = #CODE

# Train and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

# We need to split our dataset_files in train and validation files
train_files, val_files = #CODE
print(len(train_files), len(val_files))

# Now, since we created a specific SegmentationDataset class, let's use it
# we need to extract the data from train and validation files applying transformations;
# then we need to load the data with DataLoader
train_dataset= #CODE
train_loader = #CODE


val_ds = #CODE
val_loader = #CODE

# Training loop

In [ ]:
# We will need to save our model, so let's create our checkpoint_directory
checkpoint_dir = BASE_DIR / "models"
checkpoint_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
# Finally, the training loop

max_epochs = #CODE
val_interval = 1
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
#CODE
#CODE

for epoch in range(max_epochs):
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    .
    .
    .
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE
    #CODE


In [ ]:
plt.figure("train", (12, 6))
plt.subplot(1, 2, 1)
plt.title("Epoch Average Loss")
x = [i + 1 for i in range(len(epoch_loss_values))]
y = epoch_loss_values
plt.xlabel("epoch")
plt.plot(x, y)
plt.subplot(1, 2, 2)
plt.title("Val Mean Dice")
x = [val_interval * (i + 1) for i in range(len(metric_values))]
y = metric_values
plt.xlabel("epoch")
plt.plot(x, y)
plt.show()

In [ ]:
from scipy.ndimage import rotate

# Here you can load the model and check whether the predicted mask superimposes
# correctly with initial image
#CODE
#CODE

with torch.no_grad():
    for i, val_data in enumerate(val_loader):
        #CODE
        #CODE
        #CODE
        #CODE
        #CODE

        # Plot results
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.title("Original Image")
        plt.imshow()

        plt.subplot(1, 3, 2)
        plt.title("Ground Truth")
        plt.imshow()
        plt.imshow()

        plt.subplot(1, 3, 3)
        plt.title("Prediction Overlay")
        plt.imshow()
        plt.axis("off")

        plt.show()

        if i == 2:
            break